In [ ]:
import pandas as pd
import numpy as np
import scipy as sp

In [ ]:
import pbrain

In [ ]:
from pbrain import pval


In [ ]:
pval??

In [ ]:
from importlib import reload

In [ ]:
reload(pval)

In [ ]:
pwd

In [ ]:
pvals = pval.pval('','prediction_2.csv','temp_output.csv','prediction_2sub.csv',None)

In [ ]:
[np.mean(pvals < i) for i in np.linspace(0, 1, 11)]

In [ ]:
pvals = pval.pval('','prediction_2.csv','temp_output.csv',None,'reference_table.npy')

In [ ]:
[np.mean(pvals < i) for i in np.linspace(0, 1, 11)]

In [ ]:
??pval.pval

In [ ]:
df = pd.read_csv('prediction_2.csv')

In [ ]:
scores = df['score'].values

In [ ]:
test_scores = df['score'].values

In [ ]:
test_scores

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.hist(scores, bins=100)

In [ ]:
len(scores)

In [ ]:
np.diff(np.percentile(scores, [25, 75]))[0]

In [ ]:
np.logspace(-3, 0, 20)

In [ ]:
from numpy.random import shuffle
scores1 = scores.copy()
shuffle(scores1)
scores2 = scores1[int(len(scores)/2):]
scores1 = scores1[:int(len(scores)/2)]
scores2 = np.sort(scores2)

In [ ]:
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from scipy.stats import norm

iqr = sp.stats.iqr(scores1)
params = {'bandwidth': np.logspace(-2, 0, 20) * iqr}
grid = GridSearchCV(KernelDensity(), params, cv = 20)
grid.fit(scores1.reshape((-1,1)))

print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))

# use the best estimator to compute the kernel density estimate
kde = grid.best_estimator_


In [ ]:
def predict_lower_tail(scores, test_scores, bd):
    diffs = np.dot(test_scores.reshape((-1,1)), np.ones((1, len(scores)))) - np.dot(np.ones((len(test_scores), 1)), scores.reshape((1,-1)))
    pbs = np.mean(norm.cdf(diffs, loc = 0, scale = bd), 1)
    return pbs

In [ ]:
bd = grid.best_estimator_.bandwidth

In [ ]:
pvs_cv = predict_lower_tail(scores1, scores2, grid.best_estimator_.bandwidth)

In [ ]:
pvs_scott = predict_lower_tail(scores1, scores2, np.std(scores1) * len(pvs_cv)**(-0.2))

In [ ]:
pvs_silverman = predict_lower_tail(scores1, scores2, np.std(scores1) * (len(pvs_cv) * .75)**(-0.2))

In [ ]:
plt.plot(pvs_cv - np.linspace(0, 1, len(pvs_cv)))
plt.plot(pvs_scott - np.linspace(0, 1, len(pvs_cv)))
plt.plot(pvs_silverman - np.linspace(0, 1, len(pvs_cv)))


In [ ]:
np.mean((pvs_cv - np.linspace(0, 1, len(pvs_cv)))**2),\
np.mean((pvs_scott - np.linspace(0, 1, len(pvs_cv)))**2),\
np.mean((pvs_silverman - np.linspace(0, 1, len(pvs_cv)))**2)

In [ ]:
np.std(scores) * len(scores)**(-0.2)

In [ ]:
plt.plot(np.sort(pbs))

In [ ]:
np.mean(pbs < 0.05)

In [ ]:
params['bandwidth']

In [ ]:
plt.plot(params['bandwidth'], grid.cv_results_['mean_test_score'], marker = "o")

In [ ]:
np.max(np.exp(kde.score_samples(xs.reshape((-1,1)))))

In [ ]:
a,b = np.percentile(scores, [0.1, 99.9])
xs = np.linspace(a, b, 100)
plt.hist(scores, bins=100)
plt.plot(xs, 590 * 1e6 * np.exp(kde.score_samples(xs.reshape((-1,1)))))

In [ ]:
plt.plot(xs, kde.score_samples(xs.reshape((-1,1))))

In [ ]:
        # Compute lower tails on reference using KDE with Gaussian kernel
        diffs = np.dot(xs.reshape((-1,1)), np.ones((1, len(scores)))) - np.dot(np.ones((len(xs), 1)), scores.reshape((1,-1))) # a n_test x n_train matrix of differences between xs and training points
        pvals = np.mean(1-norm.cdf(diffs, loc = 0, scale = bd), 1) # the p-values obtained by averaging lower tails of all training points for each test point

In [ ]:
plt.plot(xs, np.log(pvals))

In [ ]:
ps = np.exp(kde.score_samples(scores.reshape((-1,1))))

In [ ]:
plt.plot(np.sort(ps))

In [ ]:
from scipy import interp

In [ ]:
??interp

In [ ]:
interp(10,xs, np.log(pvals))

In [ ]:

interp(3000000,xs, np.log(pvals))